In [1]:
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import train_test_split
from keras.layers import Input,Dense,Flatten,MaxPool2D,MaxPool1D,Activation,LeakyReLU,LSTM,BatchNormalization,Dropout,Conv2D,Conv1D,Lambda
from keras.models import Model,load_model
from keras.optimizers import Adam
import keras.backend as K
import h5py
from tensorflow.keras.utils import to_categorical
from keras.regularizers import l2
from sklearn.utils import shuffle

# import keras
# from keras.constraints import max_norm
# import utils

from keras.constraints import max_norm
tf.random.set_seed(42)

In [5]:
def iblock(filter_size,kernel,inputs):
    x=layers.Conv1D(filter_size,kernel,padding='same')(inputs)
#     x=layers.BatchNormalization()(x)
    x=layers.ReLU()(x)
    x=layers.Conv1D(filter_size,kernel,padding='same')(x)
#     x=layers.BatchNormalization()(x)
#     print(x.shape)
#     print(inputs.shape)
#     print(f'add x {x.shape}')
#     print(f'add inputs {inputs.shape}')
    if x.shape[-1] != inputs.shape[-1]:
        x=layers.Dense(inputs.shape[-1])(x)
    x=layers.Add()([x,inputs])
    x=layers.ReLU()(x)
#     print(x.shape)
    
    return x

def block(kernel,inputs):
    x=iblock(64,kernel,inputs)
    x=iblock(64,kernel,inputs)
    
#     print(f'block inputs {x.shape}')
    x=iblock(128,kernel,x)
#     x=iblock(128,kernel,x)
    
#     print(f'block x1 {x.shape}')
    x=iblock(256,kernel,x)
    x=iblock(256,kernel,x)
#     print(f'block x1 {x.shape}')
    x=layers.AveragePooling1D()(x)
    return x
    
def network(k1,k2,k3):
    inputs=layers.Input((1024,))
    x=layers.Reshape((inputs.shape[-1],1))(inputs)
    x=layers.Conv1D(64,7,padding='same',strides=2)(x)
#     print(x.shape)
#     x=layers.BatchNormalization()(x)
    x=layers.ReLU()(x)
    x=layers.MaxPool1D(3,2)(x)
#     print(f'max {x.shape}')
#     x7=block(k1,x)
    x5=block(k2,x)
    x5=layers.Dropout(0.2)(x5)
    x3=block(k3,x)
    x3=layers.Dropout(0.2)(x3)
    
    x=layers.Add()([x5,x3])
    x=layers.Flatten()(x)
#     x=layers.Dropout(0.4)(x)
    outputs=layers.Dense(8,activation='softmax')(x)
    model=tf.keras.Model(inputs,outputs)
    model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.001) ,metrics=['accuracy'])
    model.summary()
    return model

network=network(7,5,3)
   
    
    

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 1024)]       0           []                               
                                                                                                  
 reshape (Reshape)              (None, 1024, 1)      0           ['input_1[0][0]']                
                                                                                                  
 conv1d (Conv1D)                (None, 512, 64)      512         ['reshape[0][0]']                
                                                                                                  
 re_lu (ReLU)                   (None, 512, 64)      0           ['conv1d[0][0]']                 
                                                                                              

 re_lu_8 (ReLU)                 (None, 255, 64)      0           ['add_3[0][0]']                  
                                                                                                  
 re_lu_18 (ReLU)                (None, 255, 64)      0           ['add_8[0][0]']                  
                                                                                                  
 conv1d_9 (Conv1D)              (None, 255, 256)     82176       ['re_lu_8[0][0]']                
                                                                                                  
 conv1d_19 (Conv1D)             (None, 255, 256)     49408       ['re_lu_18[0][0]']               
                                                                                                  
 re_lu_9 (ReLU)                 (None, 255, 256)     0           ['conv1d_9[0][0]']               
                                                                                                  
 re_lu_19 

In [6]:
# tf.keras.utils.plot_model(network,show_shapes=True)

In [ ]:
my_callbacks = [
#     tf.keras.callbacks.EarlyStopping(restore_best_weights=True,patience=50),
    tf.keras.callbacks.ModelCheckpoint(filepath='network__8_class_comb_2_level_3_5.h5',save_best_only=True,monitor='val_accuracy'),
#     tf.keras.callbacks.ReduceLROnPlateau(patience=20),
]
history=network.fit(X_train,y_train,epochs=120, validation_data=(X_test,y_test), batch_size=512,callbacks=my_callbacks)

In [ ]:
cnf_matrix